<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

In [1]:
# pip install openeo==0.23.0

# 2.3 Data Access and Basic Processing

## Reduce Operators

When computing statistics over time or indices based on multiple bands, it is possible to use reduce operators.

In openEO we can use the [reduce_dimension](https://processes.openeo.org/#reduce_dimension) process, which applies a reducer to a data cube dimension by collapsing all the values along the specified dimension into an output value computed by the reducer.

Reduce the temporal dimension to a single value, the mean for instance:

In [2]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}
temporal_extent = ["2022-05-10T00:00:00Z","2022-06-30T00:00:00Z"]
bands = ["red","nir"]
datacube = local_conn.load_stac(url=url,
                                spatial_extent=spatial_extent,
                                temporal_extent=temporal_extent,
                                bands=bands)

datacube_min_time = datacube.reduce_dimension(dimension="time",reducer="min")
datacube_min_time

Did not load machine learning processes due to missing dependencies: Install them like this: `pip install openeo-processes-dask[implementations, ml]`
/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Check what happens to the datacube inspecting the resulting xArray object:

In [3]:
datacube_min_time.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-20dd979f41ea436caeeff1c742394da4' (band: 2,
                                                                y: 713, x: 1145)>
dask.array<_nanmin_skip-aggregate, shape=(2, 713, 1145), dtype=float64, chunksize=(1, 606, 860), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * band                                     (band) <U3 'red' 'nir'
  * x                                        (x) float64 6.733e+05 ... 6.848e+05
  * y                                        (y) float64 5.155e+06 ... 5.148e+06
    s2:processing_baseline                   <U5 '04.00'
    s2:saturated_defective_pixel_percentage  int64 0
    s2:datatake_type                         <U8 'INS-NOBS'
    ...                                       ...
    proj:shape                               object {10980}
    gsd                                      int64 10
    common_name                              (band) <U3 'red' 'nir'
    center_wavelength                        (band) float64 0.665 0.842
    full_width_half_max                      (band) float64 0.038 0.145
    epsg                                     int64 32632
Attributes:
    spec:                           RasterSpec(epsg=32632, bounds=(600000.0, ...
    crs:                            epsg:32632
    transform:                      | 10.00, 0.00, 600000.00|\n| 0.00,-10.00,...
    resolution:                     10.0
    reduced_dimensions_min_values:  {'time': numpy.datetime64('2022-05-11T10:...

It is possible to reduce in the same way all the available dimensions of the datacube.

We can, for instance, reduce the band dimension similarly as we did for the temporal dimension:

In [4]:
datacube_mean_band = datacube.reduce_dimension(dimension="band",reducer="mean")

The result will now contain values resulting from the average of the bands:

In [5]:
datacube_mean_band.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-d1a1f534a60005ed45a2fef6da50f6d5' (time: 21,
                                                                y: 713, x: 1145)>
dask.array<mean_agg-aggregate, shape=(21, 713, 1145), dtype=float64, chunksize=(1, 606, 860), chunktype=numpy.ndarray>
Coordinates: (12/50)
  * time                                     (time) datetime64[ns] 2022-05-11...
    id                                       (time) <U24 'S2A_32TPS_20220511_...
  * x                                        (x) float64 6.733e+05 ... 6.848e+05
  * y                                        (y) float64 5.155e+06 ... 5.148e+06
    s2:snow_ice_percentage                   (time) object 23.404773 ... 1.47...
    s2:processing_baseline                   <U5 '04.00'
    ...                                       ...
    eo:cloud_cover                           (time) float64 13.99 66.02 ... 8.2
    raster:bands                             object {'nodata': 0, 'data_type'...
    proj:transform                           object {0, 600000, 10, 5200020, ...
    proj:shape                               object {10980}
    gsd                                      int64 10
    epsg                                     int64 32632
Attributes:
    spec:                           RasterSpec(epsg=32632, bounds=(600000.0, ...
    crs:                            epsg:32632
    transform:                      | 10.00, 0.00, 600000.00|\n| 0.00,-10.00,...
    resolution:                     10.0
    reduced_dimensions_min_values:  {'band': 0}

**Quiz hint: look carefully at number of pixels of the loaded datacube!**

The reducer could be again a single process, but when computing spectral indices like NDVI, NDSI etc. an arithmentical formula is used instead.

For instance, the [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) formula can be expressed using a `reduce_dimension` process over the `bands` dimension:

$$ NDVI = {{NIR - RED} \over {NIR + RED}} $$

In [6]:
def NDVI(data):
    red = data.array_element(index=0)
    nir = data.array_element(index=1)
    ndvi = (nir - red)/(nir + red)
    return ndvi

ndvi = datacube.reduce_dimension(reducer=NDVI,dimension="band")
ndvi.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-deba0c378681cc536cde660d97b70c6e' (time: 21,
                                                                y: 713, x: 1145)>
dask.array<truediv, shape=(21, 713, 1145), dtype=float64, chunksize=(1, 606, 860), chunktype=numpy.ndarray>
Coordinates: (12/50)
  * time                                     (time) datetime64[ns] 2022-05-11...
    id                                       (time) <U24 'S2A_32TPS_20220511_...
  * x                                        (x) float64 6.733e+05 ... 6.848e+05
  * y                                        (y) float64 5.155e+06 ... 5.148e+06
    s2:snow_ice_percentage                   (time) object 23.404773 ... 1.47...
    s2:processing_baseline                   <U5 '04.00'
    ...                                       ...
    eo:cloud_cover                           (time) float64 13.99 66.02 ... 8.2
    raster:bands                             object {'nodata': 0, 'data_type'...
    proj:transform                           object {0, 600000, 10, 5200020, ...
    proj:shape                               object {10980}
    gsd                                      int64 10
    epsg                                     int64 32632
Attributes:
    spec:                           RasterSpec(epsg=32632, bounds=(600000.0, ...
    crs:                            epsg:32632
    transform:                      | 10.00, 0.00, 600000.00|\n| 0.00,-10.00,...
    resolution:                     10.0
    reduced_dimensions_min_values:  {'band': 0}

Additionally, it is possible to reduce both spatial dimensions of the datacube at the same time.

To do this, we need the `reduce_spatial` process.

We show an example where it is used to compute the standard deviation `sd`.

In [7]:
datacube_spatial_sd = datacube.reduce_spatial(reducer="sd")
datacube_spatial_sd

Verify that the spatial dimensions were collapsed:

In [8]:
datacube_spatial_sd.execute()

/home/749c4c89-469c-4d4a-9ce4-9feffea0504a/.local/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-a5c546a75307b258eb4d231743b48be0' (time: 21,
                                                                band: 2)>
dask.array<_sqrt, shape=(21, 2), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2022-05-11...
    id                                       (time) <U24 'S2A_32TPS_20220511_...
  * band                                     (band) <U3 'red' 'nir'
    s2:snow_ice_percentage                   (time) object 23.404773 ... 1.47...
    s2:processing_baseline                   <U5 '04.00'
    s2:mgrs_tile                             (time) object '32TPS' ... '32TPS'
    ...                                       ...
    proj:shape                               object {10980}
    gsd                                      int64 10
    common_name                              (band) <U3 'red' 'nir'
    center_wavelength                        (band) float64 0.665 0.842
    full_width_half_max                      (band) float64 0.038 0.145
    epsg                                     int64 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0